In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
#from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler,SMOTE, ADASYN
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,auc,roc_auc_score,precision_score,recall_score
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

In [ ]:
join = pd.read_csv('train.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
#from sklearn.utils import shuffle
#join = shuffle(join)

In [ ]:
join['leukemia'].value_counts()

In [ ]:
train['leukemia'].value_counts()

In [ ]:
for i in join.leukemia.values:
    if i  == 0:
        join.leukemia.replace(i, 'Non-Leukemia', inplace = True)
    else:
        join.leukemia.replace(i, 'Leukemia', inplace = True)

In [ ]:
join['leukemia'].value_counts()

In [ ]:
print ('Train Data Shape: {}'.format(train.shape))

print ('Test Data Shape: {}'.format(test.shape))

In [ ]:
train.isnull().sum()/len(train)*100

In [ ]:
train.corr('pearson')

In [ ]:
#train.drop(train.columns[[0,1,6,12,13,15,16]], axis = 1, inplace = True) 
train.drop(train.columns[[0]], axis = 1, inplace = True)

In [ ]:
join.drop(join.columns[[0]], axis = 1, inplace = True)

In [ ]:
join.corr('pearson')

In [ ]:
train.head()

In [ ]:
print ('Train Data Shape: {}'.format(join.shape))

In [ ]:
#test.drop(test.columns[[0,1,6,12,13,15,16]], axis = 1, inplace = True) 
test.drop(test.columns[[0]], axis = 1, inplace = True)

In [ ]:
test.head(2)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
test=np.asarray(test)
test=test.astype(np.int)
X_test = test[:,:test.shape[1]-1]
y_test = test[:,-1:]

In [ ]:
train = train.iloc[:-1, :]
train.head()

In [ ]:
train=np.asarray(train)
train=train.astype(np.int)

X_train = train[:,:train.shape[1]-1]
#X=X.astype(np.float)
#X_train=X_train.astype(np.int)
y_train = train[:,-1:]

In [ ]:
print(X_test)

In [ ]:
# **Decision Tree Classifier**

In [ ]:
from matplotlib import pyplot as plt

dtree = DecisionTreeClassifier(max_depth= 2,random_state=0)
dtree.fit(X_train,y_train)

In [ ]:
from sklearn import tree
import graphviz
# DOT data
feature_names = join.columns[:16]
target_names = join['leukemia'].unique().tolist()
dot_data = tree.export_graphviz(dtree, out_file=None, 
                                feature_names=feature_names,  
                
                                label= "all" ,
                                filled=True, impurity=False)

# Draw graph

graph = graphviz.Source(dot_data, format="png") 
graph

In [ ]:




pred = dtree.predict(X_test)
print(pred)
print(classification_report(y_test,pred))
print (confusion_matrix(y_test,pred))

precision = precision_score(y_test,pred)
recall = recall_score(y_test,pred)
print( 'precision = ', precision, '\n', 'recall = ', recall)



In [ ]:
y_pred_proba = model.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)


In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_test, pred)

In [ ]:
from sklearn import tree
model = tree.DecisionTreeClassifier()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV
param = {
    'criterion': ['entropy', 'gini'],
    'max_depth' :[2,3,4,5,7,8,9,10]
}
grid_svc = GridSearchCV(model, param_grid=param, scoring='accuracy', cv=13)

In [ ]:
grid_svc.fit(X_train, y_train)

In [ ]:
grid_svc.best_params_

In [ ]:
t= [3,5,7,8,9]
ac=[]
for x in t:
    model= tree.DecisionTreeClassifier(criterion= 'gini', max_depth= x)
    model.fit(X_train, y_train)
    ac.append(model.score(X_test, y_test))

In [ ]:
model.score(X_train, y_train)

# ****Feature Importance

In [ ]:
reg = LassoCV()
reg.fit(X_train, y_train)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X_train,y_train))
coef = pd.Series(reg.coef_, index=X_train)
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")
imp_coef = coef.sort_values()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Lasso Model")

# **#logistic Regression**

In [ ]:
#logistic Regression
log = LogisticRegression(penalty='l2', C=0.1)
log.fit(X_train,y_train)

pred = log.predict(X_test)
print(classification_report(y_test,pred))
print (accuracy_score(y_test,pred))
print (confusion_matrix(y_test,pred))

precision = precision_score(y_test,pred)
recall = recall_score(y_test,pred)
print( 'precision = ', precision, '\n', 'recall = ', recall)

y_pred_proba = log.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()
impFeatures = pd.DataFrame(log.coef_[0] ,index=train.loc[:,train.columns!='leukemia'].columns,columns=['Importance']).sort_values(by='Importance',ascending=False)
print (impFeatures)

In [ ]:
from sklearn.metrics import matthews_corrcoef, make_scorer
from sklearn.model_selection import cross_val_score
Matthew = make_scorer(matthews_corrcoef)
mean_value = np.mean(cross_val_score(model,features_norm,data.target,cv=10,scoring=Matthew))

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_test, pred)

> # Random Forest

In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
k_array = [20,50,100,120,150,200,250, 260, 280,300,1000 ]
acc=[]

ran = RandomForestClassifier(n_estimators=50,random_state=0)
ran.fit(X_test,y_test)
ran.score(X_test,y_test)
#pd.series(ran.estimator_,feature_importances_,.ravel()).hist()
pred = ran.predict(X_test)
print(classification_report(y_test,pred))
print (accuracy_score(y_test,pred))
print (confusion_matrix(y_test,pred))

precision = precision_score(y_test,pred)
recall = recall_score(y_test,pred)
print( 'precision = ', precision, '\n', 'recall = ', recall)
#joined_data
y_pred_proba = ran.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()
impFeatures = pd.DataFrame((ran.feature_importances_) ,index=train.loc[:,train.columns!='leukemia'].columns,columns=['Importance']).sort_values(by='Importance',ascending=False)
print (impFeatures)
estimator = ran.estimators_[5]

#importances = ran.feature_importances_
#final_df = pd.DataFrame({"Features": pd.DataFrame(X_train).columns, "Importances":importances})   
#final_df.set_index('importances')    
#final_df.plot.bar(color= 'teal')
#model.fit(iris.data, iris.target)
# Extract single tree

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_test, pred)

In [ ]:
k_array = [20,50,100,120,150,200,250, 260, 280,300,1000 ]
acc=[]
for i in k_array:
    rfc_100 = RandomForestClassifier(n_estimators=i, random_state=0)



# fit the model to the training set

    rfc_100.fit(X_test, y_test)



# Predict on the test set results

    y_pred_100 = rfc_100.predict(X_test)
    
    acc.append((accuracy_score(y_test, y_pred_100)))

# Check accuracy score 

    print('Model accuracy score with 100 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred_100)))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

k_array = [20,50,100,120,150,200,250, 260, 280,300,1000 ]
accuracy = [0.9212977099236641,0.9336641221374046, 0.9445648854961832,0.9441984732824428,0.9541984732824428,0.9531984732824428,0.9465648854961832,0.9501984732824428,0.9489312977099237, 0.9509312977099237,0.9509312977099237 ]

plt.plot(k_array, accuracy) 
plt.xlabel("N-Estimators") 
plt.ylabel("Accuracy") 
plt.title("Accuracy vs N-Estimators Curve Based on Train Set") 
plt.show()  # show first chart

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_test, pred)

# ****Adaboost

In [ ]:
#Adaboost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import  f1_score
clf = AdaBoostClassifier(n_estimators=300, random_state=0)
clf.fit(X_test,y_test)
y_pred=clf.predict(X_train)
print("Accuracy: ",accuracy_score(y_train, y_pred))
from sklearn.metrics import classification_report
#classification_report(y_test, y_pred)
from sklearn.metrics import precision_score
#print("Precision: ",precision_score(y_test, y_pred, average='weighted'))
from sklearn.metrics import recall_score
#print("Recall: ",recall_score(y_test, y_pred, average='weighted'))

#print (accuracy_score(y_test,pred))
print (confusion_matrix(y_test,pred))

precision = precision_score(y_test,pred)
recall = recall_score(y_test,pred)
print( 'precision = ', precision, '\n', 'recall = ', recall)

y_pred_proba = ran.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

impFeatures = pd.DataFrame((ran.feature_importances_) ,index=train.loc[:,train.columns!='leukemia'].columns,columns=['Importance']).sort_values(by='Importance',ascending=False)
print (impFeatures)

print(f1_score(y_test, y_pred, average="macro"))

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_test, y_pred)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

k_array = [20,50,100,120,150,200,250,300,1000]
accuracy = [0.9236641221374046,0.9389312977099237,0.9465648854961832, 0.9465648854961832, 0.9465648854961832, 0.9465648854961832, 0.9465648854961832, 0.9465648854961832, 0.9533898305084746 ]

plt.plot(k_array, accuracy) 
plt.xlabel("N-Estimators") 
plt.ylabel("Accuracy") 
plt.title("Accuracy vs N-Estimators Curve Based on Test Set") 
plt.show()  # show first chart 

plt.plot(k_array, accuracy, 'ro', color='black');
plt.xlabel('n_estimators')
plt.ylabel('Accuracy')

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_test, y_pred)

# ****KNN

In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import  f1_score
clf = KNeighborsClassifier(n_neighbors=10)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print("Accuracy: ",accuracy_score(y_test, y_pred))
from sklearn.metrics import classification_report
#classification_report(y_test, y_pred)
from sklearn.metrics import precision_score
print("Precision: ",precision_score(y_test, y_pred, average='weighted'))
from sklearn.metrics import recall_score
print("Recall: ",recall_score(y_test, y_pred, average='weighted'))
y_pred_proba = clf.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()
feat_importances = pd.Series(ran.feature_importances_, index=train.loc[:,train.columns!='leukemia'].columns)
feat_importances.plot(kind='barh')
F1 = 2 * (precision * recall) / (precision + recall)
print(F1)

print(f1_score(y_test, y_pred, average="macro"))

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_test, y_pred)

In [ ]:
#KNN
from sklearn.metrics import confusion_matrix
k_array = [3,5, 7, 9,11,13,15,17,19,21,23]
accuracy = []
for k in k_array:
  m = KNeighborsClassifier(n_neighbors=k)
  m.fit(X_train, y_train)
  accuracy.append(m.score(X_test,y_test))

print(accuracy)

In [ ]:
plt.plot(k_array, accuracy) 
plt.xlabel("Value of K") 
plt.ylabel("Accuracy") 
plt.title("Accuracy vs KNN Curve Based on Test Set") 
plt.show()  # show first chart

# ****Naive Baysian

In [ ]:
#Naive Baysian
from sklearn.naive_bayes  import GaussianNB
from sklearn.metrics import f1_score
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print("Accuracy: ",accuracy_score(y_test, y_pred))
from sklearn.metrics import classification_report
#classification_report(y_test, y_pred)
from sklearn.metrics import precision_score
print("Precision: ",precision_score(y_test, y_pred, average='weighted'))
from sklearn.metrics import recall_score
print("Recall: ",recall_score(y_test, y_pred, average='weighted'))
F1 = 2 * (precision * recall) / (precision + recall)
print(F1)

y_pred_proba = clf.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()
print(f1_score(y_test, y_pred, average="macro"))

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_test, y_pred)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes  import GaussianNB
clf = GaussianNB()
cross_val_score(clf, X_test,y_test, cv=10).mean()